In [ ]:
# encoding=utf-8  #or gbk 这样才能使用中文
__author__ = 'Zhangjie'

import cv2
import numpy as np
import os
#车辆识别依赖模块
from aip import AipImageClassify
#车牌识别依赖模块
import sys
import ssl
import urllib.request
import json
import string
import base64
#文件选择对话框对话框依赖模块
import time
import json
import httplib2

def Baidu_Acess():
    #百度AI个人API
    APP_ID = '11092056'
    API_KEY = 'C78v7yl1stPdGLz2gLPqZPmm'
    SECRET_KEY = '1EsdqbQi6geDdffF5uZ5KtNznNwVnw0L'

    client = AipImageClassify(APP_ID, API_KEY, SECRET_KEY)
    # client_id 为官网获取的AK， client_secret 为官网获取的SK
    host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id='+ API_KEY + '&client_secret=' + SECRET_KEY
    request = urllib.request.Request(host)
    request.add_header('Content-Type', 'application/json; charset=UTF-8')
    response = urllib.request.urlopen(request)
    #print(response)
    content = response.read()
    if (content):
        #print(content)
        data = eval(content)
        access_token = data["access_token"]
    return access_token
access_token = Baidu_Acess()

In [ ]:
# 图像路径
filepath = './'
filename = '2018-07-02 11:34:22.934533.jpg'
filepath_in = filepath + filename
#print(filepath_in)
""" 读取图片 """
def get_file_content(filePath):
    # 二进制方式打开图文件
    with open(filePath, 'rb') as fp:
        return fp.read()
    
def car_detect(filepath_in,access_token):
    image = get_file_content(filepath_in)
    # 参数image：图像base64编码
    img = base64.b64encode(image)
    # 二进制方式打开图文件
    """ 车牌识别 """
    #车牌识别api
    platerecognise_url_api = 'https://aip.baidubce.com/rest/2.0/ocr/v1/license_plate'
    platerecognise_url = platerecognise_url_api+'?access_token=' + access_token
    platerecognise_params = {"image": img}
    #params = urllib.parse.urlencode(params).encode(encoding='utf-8')
    platerecognise_params = urllib.parse.urlencode(platerecognise_params)
    platerecognise_params = platerecognise_params.encode(encoding='UTF8')

    platerecognise_request = urllib.request.Request(platerecognise_url, platerecognise_params)
    platerecognise_request.add_header('Content-Type', 'application/x-www-form-urlencoded')
    platerecognise_response = urllib.request.urlopen(platerecognise_request)
    platerecognise_content = platerecognise_response.read()
    platerecognise_content1 = platerecognise_content.decode('utf-8')
    platerecognise_json = json.loads(platerecognise_content1)
    """ 调用车辆识别 """
    cardrecognise_url_api = "https://aip.baidubce.com/rest/2.0/image-classify/v1/car"
    cardrecognise_url = cardrecognise_url_api + "?access_token=" + access_token
    cardrecognise_params = {"image":img,"top_num":5}
    cardrecognise_params = urllib.parse.urlencode(cardrecognise_params)
    cardrecognise_params = cardrecognise_params.encode(encoding='UTF8')

    cardrecognise_request = urllib.request.Request(url=cardrecognise_url, data=cardrecognise_params)
    cardrecognise_request.add_header('Content-Type', 'application/x-www-form-urlencoded')
    cardrecognise_response = urllib.request.urlopen(cardrecognise_request)
    cardrecognise_content = cardrecognise_response.read()
    cardrecognise_content1 = cardrecognise_content.decode('utf-8')
    cardrecognise_json = json.loads(cardrecognise_content1)
    if not platerecognise_content or not cardrecognise_content:
        print("false")
    else:
        print('not false')
    return platerecognise_json,cardrecognise_json